# Import Stuff

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
import typing as tp

import arch
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pyarrow
import scipy as sp
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

from ovm.garch_estimation import estimate_garch_parameters

from ovm.historical_data_io import (
    PriceHistoryColumnNames as PHCN, 
    compute_number_of_days_in_price_history, 
    compute_log_returns_from_price_history, 
    save_price_histories, 
    load_price_history
)

from ovm.utils import TimeResolution

In [3]:
%matplotlib inline

# Set Parameters

If you want to estimate and simulate a different time scale, make changes here but keep in mind that you need to download the appropriate historical data first (see data import notebook).

In [7]:
time_resolution = TimeResolution.FIFTEEN_MINUTES
directory_path = time_resolution.value

In [8]:
price_history_file_name = 'ETH-USD'

# Load Price History

In [6]:
price_history = \
    load_price_history(filename=price_history_file_name, 
                       series_name=price_history_file_name, 
                       directory_path=directory_path, 
                       period_length_in_seconds=time_resolution.in_seconds)

FileNotFoundError: 15s/ETH-USD

In [ ]:
path_length = len(price_history.price_history_df)

# Estimate GARCH Model

In [ ]:
garch_estimation_result = estimate_garch_parameters(price_history)

In [ ]:
garch_estimation_result.result.summary()

In [ ]:
garch_estimation_result.result.plot()

# Simulate New Path

In [ ]:
# This is pretty slow. I intend to speed this up later on.
garch_simulation_result = \
    garch_estimation_result.simulate(path_length=path_length, 
                                     initial_discard_length=500)

In [ ]:
garch_simulation_result.head()

This is the simulated time series

In [ ]:
plt.plot(garch_simulation_result);

Note that the GARCH model succeeds at capturing average volatility during typical times but is unable to generate a few extreme returns.

In [ ]:
plt.plot(garch_simulation_result, label='simulation');
plt.plot(price_history.unscaled_log_returns.values, label='historical data');
plt.legend();

This model mis-specification arising from the T-distribution's inability to generate sufficiently heavy tails will be addressed by the non-parametric block-bootstrap.

This can be seen in the QQ-Plot (note the outliers at the extremes)

In [ ]:
garch_estimation_result.qq_plot();